In [1]:
import requests
import sys
import time
import numpy as np
import ipywidgets as widgets
from multiprocessing import Process, Queue
from queue import Empty

This version of Big Buck Bunny is 12 min @ 1080p/30. I choose to host it at BYU because that's where ping time should be lowest for me.

In [2]:
test_url = "https://students.cs.byu.edu/~th443/bbb.mp4"

Download the video once to get its frame count:

In [3]:
# !wget 'https://students.cs.byu.edu/~th443/bbb.mp4'

Nice one-liner from https://stackoverflow.com/questions/2017843/fetch-frame-count-with-ffmpeg:

In [4]:
# !ffprobe -v error -select_streams v:0 -show_entries stream=nb_frames -of default=nokey=1:noprint_wrappers=1 -ignore_editlist 1 bbb.mp4

In [5]:
frame_count = 19036

In [6]:
fps = 30

In [7]:
minutes = frame_count / fps / 60
minutes

10.575555555555555

In [8]:
mb_count = 276134947 / (1e+6)

`frame_size` represents the average MB size of frame:

In [9]:
frame_size = mb_count / frame_count
frame_size

0.01450593333683547

`second_size` represents the average MB size of a second at 30 FPS--the minimum MB/s required for smooth playback:

In [10]:
second_size = frame_size * fps
second_size

0.4351780001050641

In [11]:
process_count = 3

In [12]:
mbps_queue = Queue()
mbps_percent_queue = Queue()

def download_measure(i):
    global mbps_list
    response = requests.get(test_url, stream=True)
    total_length = response.headers.get('content-length')

    start = time.time()
    dl = 0
    total_length = int(total_length)
    last_print_time = 0
    for data in response.iter_content(chunk_size=1024):
        dl += len(data)
        done = int(50 * dl / total_length)
        dl_mb = dl/(1e+6)
        if time.time() - last_print_time > 0.2:
            last_print_time = time.time()
            mbps_percent_queue.put((i, dl/total_length))
        
    elapsed = time.time() - start
    mbps_queue.put((total_length/1e+6)/elapsed)

In [13]:
processes = [Process(target=download_measure, args=(i,)) for i in range(process_count)]

In [14]:
progress_bars = [widgets.FloatProgress(
                        value=0,
                        min=0,
                        max=1,
                        step=0.1,
                        description=f'{i} (0%):',
                        bar_style='info',
                        orientation='horizontal'
                     )
                    for i in range(process_count)
                ]

In [15]:
def print_progress(): 
    try:
        progress_outputs = sorted([mbps_percent_queue.get(timeout=.2) for _ in processes], key=lambda a: a[0] if a else 0)
        for p in filter(lambda p: p, progress_outputs):
            progress_bars[p[0]].value = p[1]
            progress_bars[p[0]].description = f"{p[0]} ({'{:.2f}'.format(round(p[1] * 10000)/100)}%)"
    except Empty:
        pass

In [16]:
for p in processes:
    p.start()

[display(b) for b in progress_bars]
    
while True in [p.is_alive() for p in processes]:
    print_progress()

FloatProgress(value=0.0, bar_style='info', description='0 (0%):', max=1.0)

FloatProgress(value=0.0, bar_style='info', description='1 (0%):', max=1.0)

FloatProgress(value=0.0, bar_style='info', description='2 (0%):', max=1.0)

In [17]:
mbps_averages = [mbps_queue.get() for p in processes]
mbps_averages

[40.189824971798544, 39.64370020887117, 39.05855481245061]

In [18]:
mbps_average = np.average(mbps_averages)
mbps_average

39.630693331040106

In [19]:
relative_length_ratio = second_size / mbps_average
relative_length_ratio

0.010980832368234595

In [20]:
extra_length_proportion = relative_length_ratio - 1
extra_length_proportion

-0.9890191676317655

In [21]:
minutes_spent_buffering = extra_length_proportion * minutes
minutes_spent_buffering

-10.459427152799048

I got 24 minutes of buffering on a 5 MB/s connection for a 12 minute video. If this is accurate, this means I would spend twice as much time as the length of the original video just waiting for it to load--as long as there are also 3 other people streaming it.

### Potential improvements:

- Find multiple different video sources with different latencies, throughputs, and bitrates
- Figure out how to do this test continuously